In [19]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [20]:
tf.reset_default_graph()

In [21]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

def sample_z(n,m):
    return np.random.uniform(-1.,1., size=[n,m])

In [22]:
# Defining input placeholder and parameters for the generator

Z = tf.placeholder(tf.float32, shape=[None, 100])
with tf.name_scope("generator"):
    G_W1 = tf.Variable(xavier_init([100, 128]),name="G_W1")
    G_b1 = tf.Variable(tf.zeros(shape=[128]),name="G_B1")

    G_W2 = tf.Variable(xavier_init([128, 784]),name="G_W2")
    G_b2 = tf.Variable(tf.zeros(shape=[784]),name="G_B2")

theta_G = [G_W1, G_W2, G_b1, G_b2]

In [23]:
# Defining input placeholder and parameters for the discriminator

X = tf.placeholder(tf.float32, shape=[None, 784])

with tf.name_scope("discriminator"):

    D_W1 = tf.Variable(xavier_init([784, 128]), name="D_W1")
    D_b1 = tf.Variable(tf.zeros(shape=[128]),name="D_B1")

    D_W2 = tf.Variable(xavier_init([128, 1]),name="D_W2")
    D_b2 = tf.Variable(tf.zeros(shape=[1]),name="D_B2")

theta_D = [D_W1, D_W2, D_b1, D_b2]

In [24]:
def generator(z):
    with tf.name_scope("generator"):
        G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
        G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
        G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob

In [25]:
def discriminator(x):
    with tf.name_scope("discriminator"):
        D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
        D_log_prob = tf.matmul(D_h1, D_W2) + D_b2
        D_prob = tf.nn.sigmoid(D_log_prob)
    return D_prob

In [26]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [27]:
G_sample = generator(Z)
D_real = discriminator(X)
D_fake = discriminator(G_sample)

In [28]:
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

In [29]:
D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list = theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list = theta_G)

In [30]:
z_size = 100
mb_size = 16

mnist = input_data.read_data_sets('../data/MNIST_data', one_hot=True)

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [33]:
sess = tf.Session()
training_summary = tf.summary.scalar("D_loss", D_loss)
writer = tf.summary.FileWriter('summaries')
sess.run(tf.initialize_all_variables())

In [34]:
if not os.path.exists('../out/'):
    os.makedirs('../out/')

i = 0

for it in range(1000000):
    if it % 100 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_z(16, z_size)})

        fig = plot(samples)
        plt.savefig('../out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
        i += 1
        plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_mb, Z: sample_z(mb_size, z_size)})
    
    tf.summary.scalar("D_loss", D_loss_curr)
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_z(mb_size, z_size)})

    s = sess.run(training_summary, feed_dict={X:X_mb, Z: sample_z(mb_size, z_size)})
    writer.add_summary(s,it)
    if it % 100 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.368
G_loss: 2.678

Iter: 100
D loss: 0.1629
G_loss: 3.829

Iter: 200
D loss: 0.1952
G_loss: 3.315

Iter: 300
D loss: 0.2461
G_loss: 4.998

Iter: 400
D loss: 0.042
G_loss: 4.999

Iter: 500
D loss: 0.01294
G_loss: 5.859

Iter: 600
D loss: 0.01496
G_loss: 5.259

Iter: 700
D loss: 0.02053
G_loss: 5.396

Iter: 800
D loss: 0.01346
G_loss: 5.824

Iter: 900
D loss: 0.05243
G_loss: 6.636

Iter: 1000
D loss: 0.02081
G_loss: 7.069

Iter: 1100
D loss: 0.009407
G_loss: 6.475

Iter: 1200
D loss: 0.007367
G_loss: 7.118

Iter: 1300
D loss: 0.00572
G_loss: 8.164

Iter: 1400
D loss: 0.00192
G_loss: 8.059

Iter: 1500
D loss: 0.005572
G_loss: 8.459

Iter: 1600
D loss: 0.003231
G_loss: 6.295

Iter: 1700
D loss: 0.04745
G_loss: 6.282

Iter: 1800
D loss: 0.04394
G_loss: 4.187

Iter: 1900
D loss: 0.01168
G_loss: 5.748

Iter: 2000
D loss: 0.09216
G_loss: 4.639

Iter: 2100
D loss: 0.08182
G_loss: 8.338

Iter: 2200
D loss: 0.01097
G_loss: 6.542

Iter: 2300
D loss: 0.01333
G_loss: 6.556

Iter: 2

KeyboardInterrupt: 